#Convert text messages to a combined grib-files that is saved to disk

In [ ]:
# Imports
from datetime import datetime
from pathlib import Path
from src.saildoc_functions import decode_saildocs_grib_file
from src.saildoc_functions import unwrap_messages_to_payload_chunks

# ===================================================
# COPY ALL MESSAGES RECEIVED FROM INREACH
# ===================================================

TEXT_RECEIVED = """
msg 1/31
R1JJQgAAaAEAABwCYp7/gAJmAAAaARIGAAEAAAoAAAAVAAAAAAAgAP8AAAUABABV8ICzsIAAhNCAdTAPoA+gQAAAAAAAACAIAANFGOwzCAAAAggJOjpERj5w
end
msg 2/31
eH6EeZCaoZ2IADc3NzdHUklCAABoAQAAHAJinv+AAmYAABoBEgYAAQAMCgAAABUAAAAAACAA/wAABQAEAFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAIAgAA0UY
end
msg 3/31
5rIIAAoMDA4/RElQRnKBiolwlqaqoIoANzc3N0dSSUIAAGgBAAAcAmKe/4ACZgAAGgESBgABABgKAAAAFQAAAAAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6AP
end
msg 4/31
oEAAAAAAAAAgCAADRRjpnggACg4KBDxISEIybn2EeGeRoqKWfgA3Nzc3R1JJQgAAaAEAABwCYp7/gAJmAAAaARIGAAEAJAoAAAAVAAAAAAAgAP8AAAUABABV
end
msg 5/31
8ICzsIAAhNCAdTAPoA+gQAAAAAAAACAIAANFGOpiCAIGCQgAO0BBPjJneHp2bnaSmJOAADc3NzdHUklCAABoAQAAHAJinv+AAmYAABoBEgYAAQAwCgAAABUA
end
msg 6/31
AAAAACAA/wAABQAEAFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAIAgAA0UY6K0IAggKBQA7QkM9NmxydHBodoaJhHcANzc3N0dSSUIAAGYBAAAcAmKe/4ACZgAA
end
msg 7/31
GgESBgABADwKAAAAFQAAAAAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAeDAADRRjr1AcAGDhgjiFCfvsGzhx1ZvH/7uAANzc3N0dSSUIAAGYB
end
msg 8/31
AAAcAmKe/4ACZgAAGgESBgABAEgKAAAAFQAAAAAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAeDAADRRjpsgcAJIESbiBEiRL+bZqxYNvLXcAA
end
msg 9/31
Nzc3N0dSSUIAAGYBAAAcAmKe/4ACZgAAGgESBgABAFQKAAAAFQAAAAAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAeDAADRRjtQwcACCBgi5k2
end
msg 10/31
cNKFWtWlyqdWhAAANzc3N0dSSUIAAGYBAAAcAmKe/4ACZgAAGgESBgABAGAKAAAAFQAAAAAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAeDAAD
end
msg 11/31
RRjvkQcAGCBwK5k0bK6NSlPgTpkh8uAANzc3N0dSSUIAAGgBAAAcAmKe/4AhaQAKGgESBgABAAAKAAAAFQAAAQAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6AP
end
msg 12/31
oEAAAAAAAAAgCAAAwnrcqAgSDBAAAhwOEBQjJCclKVBgaVl+iwA3Nzc3R1JJQgAAZgEAABwCYp7/gCFpAAoaARIGAAEADAoAAAAVAAABAAAgAP8AAAUABABV
end
msg 13/31
8ICzsIAAhNCAdTAPoA+gQAAAAAAAAB4MAADCfEBSBzZIIAOlRgg0uYMTz6BvxT9XIAA3Nzc3R1JJQgAAZgEAABwCYp7/gCFpAAoaARIGAAEAGAoAAAAVAAAB
end
msg 14/31
AAAgAP8AAAUABABV8ICzsIAAhNCAdTAPoA+gQAAAAAAAAB4MAADCemZgBw4AYOQDQZBEkTpF7VzQpaO2kAA3Nzc3R1JJQgAAZgEAABwCYp7/gCFpAAoaARIG
end
msg 15/31
AAEAJAoAAAAVAAABAAAgAP8AAAUABABV8ICzsIAAhNCAdTAPoA+gQAAAAAAAAB4MAADCa+PsBwAAOKOiAosyYZrEapn44XMGEAA3Nzc3R1JJQgAAaAEAABwC
end
msg 16/31
Yp7/gCFpAAoaARIGAAEAMAoAAAAVAAABAAAgAP8AAAUABABV8ICzsIAAhNCAdTAPoA+gQAAAAAAAACAIAADCdd66CAEABxEUBwMRGBo3Qjg5RI2VdomEADc3
end
msg 17/31
NzdHUklCAABoAQAAHAJinv+AIWkAChoBEgYAAQA8CgAAABUAAAEAACAA/wAABQAEAFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAIAgAAMJ6KlYIAAkJCxcLERkX
end
msg 18/31
IUdFQEtZnKejoZoANzc3N0dSSUIAAGgBAAAcAmKe/4AhaQAKGgESBgABAEgKAAAAFQAAAQAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAgCAAA
end
msg 19/31
wnmY/AgCAAMBAiIcJSItT1NERmWrpLK5uQA3Nzc3R1JJQgAAaAEAABwCYp7/gCFpAAoaARIGAAEAVAoAAAAVAAABAAAgAP8AAAUABABV8ICzsIAAhNCAdTAP
end
msg 20/31
oA+gQAAAAAAAACAIAADCae3GCAsGAgUAICQhIyxaS0FHV5y0qaGnADc3NzdHUklCAABoAQAAHAJinv+AIWkAChoBEgYAAQBgCgAAABUAAAEAACAA/wAABQAE
end
msg 21/31
AFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAIAgAAMJmZvYIAAMOCQwaDRYcJz5EQ1N0Yquzs7IANzc3N0dSSUIAAGYBAAAcAmKe/4AiaQAKGgESBgABAAAKAAAA
end
msg 22/31
FQAAAQAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAeDAAAwlMm6geO+kK00yA1TCb9aQgAd/3xsDAANzc3N0dSSUIAAGgBAAAcAmKe/4AiaQAK
end
msg 23/31
GgESBgABAAwKAAAAFQAAAQAAIAD/AAAFAAQAVfCAs7CAAITQgHUwD6APoEAAAAAAAAAgCAAAwlZ9cghORkIkF0tELyYNY1A7GgCKc0QbHQA3Nzc3R1JJQgAA
end
msg 24/31
ZgEAABwCYp7/gCJpAAoaARIGAAEAGAoAAAAVAAABAAAgAP8AAAUABABV8ICzsIAAhNCAdTAPoA+gQAAAAAAAAB4MAADCPsmEB2a8qNCOkw8aAmzl5gbsuUC6
end
msg 25/31
8AA3Nzc3R1JJQgAAaAEAABwCYp7/gCJpAAoaARIGAAEAJAoAAAAVAAABAAAgAP8AAAUABABV8ICzsIAAhNCAdTAPoA+gQAAAAAAAACAIAADCRiHKCDgyIBgS
end
msg 26/31
NiYrGBNhUjQNAIJ3OCcUADc3NzdHUklCAABmAQAAHAJinv+AImkAChoBEgYAAQAwCgAAABUAAAEAACAA/wAABQAEAFXwgLOwgACE0IB1MA+gD6BAAAAAAAAA
end
msg 27/31
HgwAAMIu164HSkxoIUfRFAAyhPRkhW/JEOEAADc3NzdHUklCAABmAQAAHAJinv+AImkAChoBEgYAAQA8CgAAABUAAAEAACAA/wAABQAEAFXwgLOwgACE0IB1
end
msg 28/31
MA+gD6BAAAAAAAAAHgwAAMI1VNoHTGxgAM2PEA5ia6WECHaa5NCgADc3NzdHUklCAABmAQAAHAJinv+AImkAChoBEgYAAQBICgAAABUAAAEAACAA/wAABQAE
end
msg 29/31
AFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAHgwAAMIrCLgHTlxIAM1THVhuvMaGx+ey4bEQADc3NzdHUklCAABmAQAAHAJinv+AImkAChoBEgYAAQBUCgAAABUA
end
msg 30/31
AAEAACAA/wAABQAEAFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAHgwAAMIvKjgHVnQAwEyOmy4di3Slgk9+6LCAADc3NzdHUklCAABiAQAAHAJinv+AImkAChoB
end
msg 31/31 
EgYAAQBgCgAAABUAAAEAACAA/wAABQAEAFXwgLOwgACE0IB1MA+gD6BAAAAAAAAAGgAAAMI3lS4GWbbPLdZjSQ5vuUAe1vkfNzc3Nw==
end
"""

# ===================================================
# UNWRAP TEXT_RECEIVED
# ===================================================

payload_chunks = unwrap_messages_to_payload_chunks(TEXT_RECEIVED)

# ===================================================
# DECODE PAYLOAD MESSAGES
# ===================================================

grib_bytes = decode_saildocs_grib_file(payload_chunks)

print(f"Decoded GRIB size: {len(grib_bytes)} bytes")

# ===================================================
# WRITE FILE TO DISK
# ===================================================

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = Path(f"./decoded_grib_{current_time}.grb")

with output_path.open("wb") as f:
    f.write(grib_bytes)

print(f"Decoded GRIB saved to: {output_path.resolve()}")


Decoded GRIB size: 2776 bytes
Decoded GRIB saved to: /Users/philipjuhl/Projects/weather-grib-mail-functions/decoded_grib_20260118_204636.grb


#Original script